Ben Goodwin

NLP Homework 2

1/24/2022

1.	In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. (Various methods will be discussed in the live session.)

In [32]:
#Bring the libraries in
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from sklearn import preprocessing
import numpy as np
#nltk.download()

In [74]:
#Bring some texts in to compare and play with
#These are the same texts I used for homework 1, with the addtion of a fourth text

#Book title, Old-time Stories, Fairy Tales and Myths Retold by Children
raw = open('talesandmyths.txt').read()
raw = word_tokenize(raw)
#print(lexical_diversity(raw)) #Fails long word method!

#Book title, McGuffey's Fifth Eclectic Reader by William Holmes McGuffey
raw1 = open('fifthelectronicreader.txt').read()
raw1 = word_tokenize(raw1)

#Book title, The Literary World Seventh Reader by Browne, Metcalf, and Withers
raw2 = open('litearyworld.txt').read()
raw2 = word_tokenize(raw2)

#Test book, new text for this week
raw3 = open('nature.txt').read()
raw3 = word_tokenize(raw3)


0.12031294723785899


In [48]:
#This method scores the vocab size of a text
#Input: Text
#Return: score
def score_vocab_size(text):
    #Handles computing the vocab for the text
    vocab = len(set(word.lower() for word in raw if word.isalpha()))
    #Builds an array of vocabs based on history, plus the text passed to the function
    x_array = np.array([969,10611,9708,vocab])
    #Call to normalization fuction for 0,1 scale
    normalized_arr = preprocessing.normalize([x_array])
    #Get the score for the texted passed to the function
    score = normalized_arr[0,3]
    return score


This was an interesting problem that took a bit of thought.  I finally realized that normalization takes some previous data to scale the new input to.  For this function I took computed the vocab score for 3 other texts plus the text in question and used these as data points.  Then using the preprocessing.normalize function which behind the scenes computes: (zi = (xi – min(x)) / (max(x) – min(x))).  We then have a 0-1 scale built off of our data. The function then looks up the 4th entry in the array and returns the score of the vocab.

2.	After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.

In [62]:
#Input: text
#Return: vocab_score/long_word_score (word_length >15)

def long_word_score(text):
    #Call vocab_score method on text
    score = vocabulary_score(text)
    #Iterate over text and determine which word_length >15
    V = set(text)
    long_words = [w for w in V if len(w) > 15]
    #Length of list of long_words
    long_words_len = len(long_words)
    #Develop array with scores
    x_array=np.array([539.3,353.7,539.34,score/long_words_len])
    #Normalize data
    normalized_arr = preprocessing.normalize([x_array])
    #Retreive function input text
    score = normalized_arr[0,3]
    return score

I actually solved this problem before I did number 1, I was initially confused by the term "vocab score".  I finally realized that normalization takes some previous data to scale the new input to.  For this function the idea of scaling is the same as above with another step.  In this function I consider vocab long if its greater than 15 characters.  I then determine the score by dividing the vocab score by the long word score (so the vocab score over the score of long words, essentially a metric of how many words are long words relative to the vocabulary.). The normalization follows suit, where the three training texts determine the scale, and then the fourth text is added and computed.  Finally the fourth element of the score array is returned.

3.	Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts you used in homework 1

In [79]:
def lexical_diversity(text):
    #Compute lexical diversity
    lex_diversity = len(set(text)) / len(text)
    #Build array with data plus data from function input
    x_array=np.array([0.112959,0.12031294,0.0944273,lex_diversity])
    #Normalize the data 
    normalized_arr = preprocessing.normalize([x_array])
    #Retrieve function input text
    score = normalized_arr[0,3]
    return score

In [88]:
def text_difficulty_score(text):
    #Call lexical diversity function
    diversity = lexical_diversity(text)
    #Call vocab score function
    vocab_score= score_vocab_size(text)
    #Call long word score
    long_word_Score = long_word_score(text)
    #Compute total score with equal weighting
    total_score = (diversity*0.33333)+(vocab_score*0.33333)+(long_word_Score*0.33333)
    return total_score

In [104]:
#Test difficulty score (raw3 should be highest)
print(text_difficulty_score(raw3))

#Test difficulty score (raw1 should be lowest)
print(text_difficulty_score(raw1))

0.4114005240100155
0.3186533674139358


Luckily as expected the difficulty score is the highest for the fourth text (raw3).  This means to me that the function is working as expected.  

The text difficulty score calls the lexical diversity function, vocab_score function, and long_word_score function and then computes a weighted average of each.  The total score is computed by multiplying each score by 3 and adding it to the other scores.  When applied to the texts from homework 1, the algorithms perform as expected with the first book ranking the easiest with the lowest difficulty score, and the latest book (not a childrens book) ranking the hardest.